In [1]:
from cv2 import normalize
import numpy as np
import cv2 as cv
import math
import time
import re
import os

In [2]:
def filter_pose_mask(array, mask):
    return [element for (i, element) in enumerate(array) if mask[i][0] == 1]

In [3]:
bf = cv.BFMatcher(cv.NORM_L1)

def featurematch_relloc(ref_data, query_data, camera_mtx, draw=False):
    (ref_img, (ref_kp, ref_desc)) = ref_data
    (query_img, (query_kp, query_desc)) = query_data

    # Match descriptors.
    matches = bf.knnMatch(ref_desc, query_desc, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) < 5:
        return None, None

    good_ref_points = np.zeros(shape=(len(good_matches), 2), dtype=np.float32)
    good_query_points = np.zeros(shape=(len(good_matches), 2), dtype=np.float32)
    for i in range(len(good_matches)):
        match = good_matches[i]
        good_ref_points[i, 0] = ref_kp[match.queryIdx].pt[0]
        good_ref_points[i, 1] = ref_kp[match.queryIdx].pt[1]
        good_query_points[i, 0] = query_kp[match.trainIdx].pt[0]
        good_query_points[i, 1] = query_kp[match.trainIdx].pt[1]

    E, E_mask = cv.findEssentialMat(good_query_points, good_ref_points, camera_mtx, method=cv.RANSAC, prob=0.99, threshold=0.1, mask=None)
    num_good, delta_R, delta_t, pose_mask = cv.recoverPose(E, good_query_points, good_ref_points, cameraMatrix=camera_mtx, mask=E_mask)

    if draw:
        img = cv.drawMatches(ref_img,ref_kp,query_img,query_kp,filter_pose_mask(good_matches, pose_mask),None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS) 
        cv.imshow('matches', img)
        cv.waitKey()

    if num_good >= 8:
        return delta_t, delta_R
    else:
        return None, None

In [4]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

# v1 and v2 must be normalized
def angle_between(v1, v2):
    return np.arccos(np.clip(np.dot(v1, v2), -1.0, 1.0))

def signed_angle_between(v1, v2):
    return math.atan2(v1[0]*v2[1] - v1[1]*v2[0], v1[0]*v2[0] + v1[1]*v2[1]);

In [5]:
def rotate_vector(v, r):
    rot = np.array([[math.cos(r), -math.sin(r)], [math.sin(r), math.cos(r)]])
    v_rot = np.dot(rot, v)
    return v_rot

In [6]:
def get_direction(frame_with_points, ref_idx, query_idx, angles, mtx):
    t1, r1 = featurematch_relloc(frame_with_points[ref_idx], frame_with_points[query_idx], mtx, False)
    if t1 is None or r1 is None:
        return None, None
    v = (t1[[0, 2], 0])
    ref_r = angles[ref_idx]
    v_rot = rotate_vector(v, ref_r)
    return v_rot, r1


In [7]:
def compass_heading_to_angle(heading):
    # North is -135.6 degrees from the world x-axis
    # 0 degrees from the camera's perspective is straight forward along the camera y-axis so another 90 degrees to convert to camera x-axis
    correction_degrees = 135.6 + 90
    correction = np.deg2rad(correction_degrees)
    angle = heading - correction
    if angle < 0:
        angle = 2 * np.pi + angle
    return angle

In [8]:
def protractor_reading_to_angle(reading):
    # Protractor +tive is clock wise on the grid
    # Protractor 0 degrees is -90 degrees from the world x-axis
    # 0 degrees from the camera's perspective is straight forward along the camera y-axis so another 90 degrees to convert to camera x-axis
    reading = -reading
    reading = reading - (90 + 90)
    return np.deg2rad(reading) + 2 * math.pi

In [9]:
with np.load('calibration.npz') as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

In [10]:
base = "./protractor_positioned/"
paths = os.listdir(base)

angles = []
tagged_angles = []
positions = []
for path in paths:
    result = re.search("x(\d*\.\d*)_y(\d*\.\d*)_ch(\d*\.\d*)_t(-?\d*\.?\d*).png", path)
    if result:
        positions.append(np.array([float(result.group(1)), float(result.group(2))]))
        angles.append(compass_heading_to_angle(float(result.group(3))))
        tagged_angles.append(protractor_reading_to_angle(float(result.group(4))))
    else:
        print("Missed one")

frames = [cv.cvtColor(cv.imread(base + path), cv.COLOR_BGR2GRAY) for path in paths]

first_frame = frames[0]
h, w = first_frame.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

undistorted = [cv.undistort(frame, mtx, dist, None, newcameramtx) for frame in frames]

detector = cv.ORB_create(10000)
descriptor = cv.xfeatures2d.BEBLID_create(0.75)

frame_with_points = [(frame, descriptor.compute(frame, detector.detect(frame, None))) for frame in undistorted]

In [11]:
paths

['x90.0_y60.0_ch1.127_t-22.5.png',
 'x90.0_y60.0_ch0.810_t-11.25.png',
 'x120.0_y60.0_ch6.259_t22.5.png',
 'x90.0_y60.0_ch0.276_t11.25.png',
 'x120.0_y60.0_ch0.996_t-11.25.png',
 'x90.0_y60.0_ch0.579_t0.png',
 'x90.0_y120.0_ch0.184_t22.5.png',
 'x120.0_y60.0_ch0.736_t-11.25.png',
 'x90.0_y60.0_ch1.109_t-22.5.png',
 'x90.0_y120.0_ch1.073_t-22.5.png',
 'x120.0_y60.0_ch0.504_t0.png',
 'x120.0_y90.0_ch0.790_t-11.25.png',
 'x120.0_y60.0_ch0.420_t22.5.png',
 'x90.0_y120.0_ch0.849_t-11.25.png',
 'x90.0_y60.0_ch0.739_t0.png',
 'x120.0_y60.0_ch1.144_t-22.5.png',
 'x120.0_y120.0_ch0.903_t-11.25.png',
 'x120.0_y90.0_ch1.042_t-22.5.png',
 'x120.0_y120.0_ch1.121_t-22.5.png',
 'x120.0_y90.0_ch0.259_t11.25.png',
 'x120.0_y120.0_ch0.693_t0.png',
 'x90.0_y60.0_ch0.921_t-11.25.png',
 'x90.0_y60.0_ch6.278_t22.5.png',
 'x120.0_y90.0_ch0.496_t0.png',
 'x90.0_y120.0_ch0.426_t11.25.png',
 'x90.0_y60.0_ch0.522_t11.25.png',
 'x90.0_y60.0_ch0.263_t22.5.png',
 'x120.0_y60.0_ch0.819_t0.png',
 'x120.0_y60.0_ch0.58

In [12]:
import math

def recover_euler_angles(R):
    if R[2][0] != 1 and R[2][0] != -1:
        theta_1 = -math.asin(R[2][0])
        theta_2 = math.pi - theta_1

        psi_1 = math.atan2(R[2][1]/math.cos(theta_1), R[2][2]/math.cos(theta_1))
        psi_2 = math.atan2(R[2][1]/math.cos(theta_2), R[2][2]/math.cos(theta_2))

        phi_1 = math.atan2(R[1][0]/math.cos(theta_1), R[0][0]/math.cos(theta_1))
        phi_2 = math.atan2(R[1][0]/math.cos(theta_2), R[0][0]/math.cos(theta_2))

        return [(psi_1, theta_1, phi_1), (psi_2, theta_2, phi_2)]
    else:
        phi = 0
        if R[2][0] == -1:
            theta = math.pi / 2
            psi = phi + math.atan2(R[0][1], R[0][2])
        else:
            theta = -math.pi / 2
            psi = -phi + math.atan2(-R[0][1], -R[0][2])
        return [(psi, theta, phi)]


In [13]:
def get_angle_between_frames(frame_with_points, ref, query, angles, mtx):
    try:
        d, r = get_direction(frame_with_points, ref, query, angles, mtx)
        if r is not None:
            v_z = [0, 0, 1]
            v_rot = np.dot(v_z, r)

            v2_y = [0, 1]
            v2_rot = unit_vector([v_rot[0], v_rot[2]])

            return signed_angle_between(v2_y, v2_rot)
        else:
            return None
    except:
            return None

In [64]:
errors = []
total = 0
failed = 0

for i in range(0, len(paths)):
    for j in range(i, len(paths)):
        total = total + 1
        true_diff = tagged_angles[j] - tagged_angles[i]

        try:
            d, r = get_direction(frame_with_points, i, j, angles, mtx)
            if r is not None:
                v_z = [0, 0, 1]
                v_rot = np.dot(v_z, r)

                v2_y = [0, 1]
                v2_rot = unit_vector([v_rot[0], v_rot[2]])

                estimate_deff = np.rad2deg(signed_angle_between(v2_y, v2_rot))

                error = true_diff - estimate_deff

                #if error > 100:
                #    print(error)
                #    featurematch_relloc(frame_with_points[j], frame_with_points[i], mtx, True)

                errors.append(error)
            else:
                failed = failed + 1
        except:
            failed = failed + 1

In [14]:
abs_errors = [np.abs(error) for error in errors]

NameError: name 'errors' is not defined

In [15]:
np.percentile(abs_errors, [68, 95, 99.7])

NameError: name 'abs_errors' is not defined

In [16]:
failed / total

NameError: name 'failed' is not defined

In [17]:
import matplotlib.pyplot as plt

plt.hist(errors, bins=50)

NameError: name 'errors' is not defined

In [18]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

data = errors

# Fit a normal distribution to the data:
mu, std = norm.fit(data)

# Plot the histogram.
plt.hist(data, bins=50, density=True, alpha=0.6, color='g')

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, 10)
plt.plot(x, p, 'k', linewidth=2)
title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
plt.title(title)

plt.show()

NameError: name 'errors' is not defined

In [19]:
compass_errors = []

for i in range(0, len(angles)):
    a = angles[i]
    ta = tagged_angles[i]
    compass_errors.append(np.rad2deg(ta - a))

abs_compass_errors = [np.abs(error) for error in compass_errors]

In [20]:
np.percentile(abs_compass_errors, [68, 95, 99.7])

array([12.43629029, 22.51991666, 24.38447439])

In [48]:
import scipy.stats

def probability_of_value(value, dist):
    norm = scipy.stats.norm(dist[0], dist[1])
    low = norm.cdf(value - np.deg2rad(2.5))
    high = norm.cdf(value + np.deg2rad(2.5))
    return high - low

In [69]:
import numpy as np

def naive_gradient(func, p, epsilon):
    p_diff = p.copy()
    print(f"Val: {func(p_diff)}")
    grad = np.zeros(shape=(p.shape[0]))
    for dim in range(0, p.shape[0]):
        p_diff[dim] = p_diff[dim] + epsilon
        high_val = func(p_diff)
        p_diff[dim] = p_diff[dim] - 2. * epsilon
        low_val = func(p_diff)
        grad[dim] = (high_val - low_val) / (2 * epsilon)
    return grad

In [101]:
import random

headings = [[angle, np.deg2rad(12)] for angle in angles]

diffs = {}
for i in range(0, len(headings)):
    refs = [j for j in range(0, len(headings)) if j is not i]
    for j in refs:
        angle = get_angle_between_frames(frame_with_points, i, j, angles, mtx)
        if angle is not None:
            diffs[(i, j)] = [angle, np.deg2rad(10)]

In [102]:
def probability_of_estimates(estimates):
    probabilities = []
    for (i, heading_1) in enumerate(headings):
        estimate = estimates[i]
        probabilities.append(probability_of_value(estimate, heading_1))
        for (j, heading_2) in enumerate(headings):
            if i is not j:
                key = (j, i)
                if key in diffs:
                    diff = diffs[key]
                    diff_heading = heading_2 + diff
                    probabilities.append(probability_of_value(estimate, diff_heading))
    return np.mean(probabilities)

def gradient_of_estimates(estimates):
    return naive_gradient(probability_of_estimates, estimates, 0.1)

In [103]:
def gradient_ascent(gradient, start, learn_rate, n_iter=50, tolerance=1e-06):
    vector = start
    for _ in range(n_iter):
        grad = gradient(vector)
        diff = grad * learn_rate
        print(diff)
        vector += diff
    print(f"{n_iter} iterations")
    return vector

In [104]:
optimized_headings = gradient_ascent(gradient_of_estimates, np.array(angles), 5, 50)

Val: 0.07950535524749366
[-0.02524066 -0.00618498  0.01801765  0.01892832 -0.00921267 -0.00335612
  0.02226264  0.01351755 -0.02186731 -0.02453439  0.01128775  0.00402819
  0.01521552 -0.0026605  -0.01120824 -0.02499152 -0.00592283 -0.01278189
 -0.01622073  0.02085573  0.00020292 -0.01976645  0.0146717   0.01283519
  0.00655847  0.00437853  0.01472785 -0.00661473  0.00754338  0.00289887
 -0.01608494  0.01693357]
Val: 0.08088552433339832
[-0.02473478 -0.00554442  0.01836504  0.01821369 -0.00815755 -0.00314903
  0.02168227  0.01224907 -0.02076126 -0.0216112   0.01102889  0.00366637
  0.01422193 -0.00237565 -0.01037764 -0.02295249 -0.00530656 -0.01101713
 -0.01441862  0.02012432  0.00019154 -0.0180066   0.01461773  0.01209587
  0.00604251  0.0039767   0.01348343 -0.00606116  0.00704028  0.00269978
 -0.01385313  0.01643507]
Val: 0.08209683510422609
[-0.02404424 -0.00497745  0.01868003  0.01744019 -0.00722064 -0.00295551
  0.02098989  0.0110455  -0.01953832 -0.01893109  0.01077205  0.003333

In [105]:
optimized_errors = []

for i in range(0, len(optimized_headings)):
    a = optimized_headings[i]
    ta = tagged_angles[i]
    optimized_errors.append(np.rad2deg(ta - a))

abs_optimized_errors = [np.abs(error) for error in optimized_errors]

In [106]:
np.percentile(abs_optimized_errors, [68, 95, 99.7])

array([11.28406465, 20.09082448, 28.89202281])

In [107]:
errors = []

for i in range(0, len(headings)):
    a = angles[i]
    ta = tagged_angles[i]
    errors.append(np.rad2deg(ta - a))

abs_errors = [np.abs(error) for error in errors]

In [108]:
np.percentile(abs_errors, [68, 95, 99.7])

array([12.43629029, 22.51991666, 24.38447439])